In [1]:
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm
np.random.seed(42)

In [2]:
df = pd.read_csv("../data/training_sample.csv")
df.head(3)

,unitID,weekID,outcome,treatment,X1,X2,X3,C1,C2,C3
0,UNIT01155,0,470,0.0,64.225447,88362,0,M,KF7,E_2
1,UNIT01155,1,534,0.0,64.225447,87892,0,M,KF7,E_2
2,UNIT01155,2,550,0.0,64.225447,87358,0,M,KF7,E_2


Covariates
1. X1 - static cts
2. X2 - temporal cts
3. X3 - temporal binary
4. C1 - static categorical (15 levels)
5. C2 - (Discard) static categorical (2495 levels)
6. C3 - static categorical (6 levels)

# Preprocessing

In [3]:
df = pd.concat([df, pd.get_dummies(df["C3"], drop_first=True)], axis=1).drop("C3", axis=1)
df['outcome'] = np.log(df['outcome'] + 1) # Deskew and bring to same scale as other covariates
df['X1'] = np.log(df['X1'])
df['X2'] = np.log(df['X2'])
df = df.drop(["C1", "C2"], axis=1)
df.head()

,unitID,weekID,outcome,treatment,X1,X2,X3,E_2,E_3,E_4,E_5,E_6
0,UNIT01155,0,6.154858,0.0,4.1624,11.389197,0,1,0,0,0,0
1,UNIT01155,1,6.282267,0.0,4.1624,11.383864,0,1,0,0,0,0
2,UNIT01155,2,6.311735,0.0,4.1624,11.377770,0,1,0,0,0,0
3,UNIT01155,3,6.333280,0.0,4.1624,11.371454,0,1,0,0,0,0
4,UNIT01155,4,7.393878,0.1,4.1624,11.364959,1,1,0,0,0,0


# Cross-validation split (Train 80 - Val 10 -Test 10)

In [4]:
# Train - Val - Test split
units = df['unitID'].unique()
num_units = len(units)
cnt_train_units = int(0.8 * num_units)
cnt_val_units = int(0.1 * num_units)
cnt_test_units = num_units - cnt_train_units - cnt_val_units

train_units = np.random.choice(units, size=cnt_train_units, replace=False)
val_units = np.random.choice(list(set(units)-set(train_units)), size=cnt_val_units, replace=False)
test_units = list(set(units)-set(train_units)-set(val_units))

train = df[df['unitID'].isin(train_units)].sort_values(by=['unitID', 'weekID'])
val = df[df['unitID'].isin(val_units)].sort_values(by=['unitID', 'weekID'])
test = df[df['unitID'].isin(test_units)].sort_values(by=['unitID', 'weekID'])

In [5]:
len(train_units), len(val_units), len(test_units) # No. of units in each set

(3126, 390, 392)

In [6]:
len(train), len(val), len(test) # Size of each set

(296970, 37050, 37240)

### Saving Data

In [7]:
train.to_csv("../data/train.csv", index=False)
val.to_csv("../data/val.csv", index=False)
test.to_csv("../data/test.csv", index=False)

# Data Preparation for CRN

## Keys required in Dataset object
1. current_covariates
2. current_treatments
3. previous_treatments
4. outputs
5. active_entries
6. sequence_lengths

# Encoder

In [8]:
# Modeling parameters
num_time_steps = 90
num_treatments = 6
num_outputs = 1
horizon = 1 # Output (horizon:t)
offset = 1 # Covariates (1:t-offset)
input_features_enc = ['outcome', 'X1', 'X2', 'X3', 'E_2', 'E_3', 'E_4', 'E_5', 'E_6']

In [9]:
def process_data_encoder(df, num_time_steps=90):
    df = df[df['weekID'] < num_time_steps]
    cnt_units = df['unitID'].nunique()

    current_covariates = df[input_features_enc].values.reshape(cnt_units, num_time_steps, len(input_features_enc))
    current_covariates = current_covariates[:, :-offset, :] # (num_units, 1-94 timesteps, num_input_features)

    current_treatments = pd.get_dummies(
                            df['treatment']).values.reshape(cnt_units, num_time_steps, num_treatments)
    current_treatments = current_treatments[:, :-offset, :] # (num_units, 1-94, 6). One-Hot-encoded treatments
    previous_treatments = current_treatments[:, :-1, :] # (num_units, 1-93, 6)

    outputs = df['outcome'].values.reshape(cnt_units, num_time_steps, num_outputs) 
    outputs = outputs[:, horizon:, :] # (num_units, 2-95 timesteps, 1)

    active_entries = np.ones((cnt_units, num_time_steps-1, 1)) # Each unit has data for all time steps
    sequence_lengths = (num_time_steps-1) * np.ones(cnt_units)
    
    data = {"current_covariates": current_covariates, 
            "current_treatments": current_treatments,
            "previous_treatments": previous_treatments,
            "outputs": outputs,
            "active_entries": active_entries,
            "sequence_lengths": sequence_lengths
           }
    return data

In [10]:
train_obj_enc = process_data_encoder(train)
val_obj_enc = process_data_encoder(val)
test_obj_enc = process_data_encoder(test)

### Export pickle files

In [11]:
with open('../data/train_enc.p', 'wb') as f:
    pickle.dump(train_obj_enc, f, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('../data/val_enc.p', 'wb') as f:
    pickle.dump(val_obj_enc, f, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('../data/test_enc.p', 'wb') as f:
    pickle.dump(test_obj_enc, f, protocol=pickle.HIGHEST_PROTOCOL)

In [12]:
for k in train_obj_enc.keys():
    print(k, train_obj_enc[k].shape)

current_covariates (3126, 89, 9)
current_treatments (3126, 89, 6)
previous_treatments (3126, 88, 6)
outputs (3126, 89, 1)
active_entries (3126, 89, 1)
sequence_lengths (3908,)


# Decoder

In [13]:
# Modeling parameters
projection_horizon = 5 # Predict outcomes for next 5 time steps
num_time_steps_dec = num_time_steps + projection_horizon # 95
input_features_dec = ['outcome', 'X1', 
                      'E_2', 'E_3', 'E_4', 'E_5', 'E_6'] # Outcome (should be 1st variable) + Static variables
feat_idx = [0,1,4,5,6,7,8] # Index of above features in original df (Outcome + Static)

In [14]:
def process_train_val_decoder(obj_enc):
    obj_enc['current_covariates'] = obj_enc['current_covariates'][:, :, feat_idx]
    return obj_enc

In [15]:
def process_test_decoder(df, num_time_steps_dec=95):
    df = df[df['weekID'] < num_time_steps_dec]
    cnt_units = df['unitID'].nunique()

    current_covariates = df[input_features_dec].values.reshape(cnt_units, num_time_steps_dec, len(input_features_dec))
    current_covariates = current_covariates[:, :-offset, :] # (num_units, 1-94 timesteps, num_input_features)

    current_treatments = pd.get_dummies(
                            df['treatment']).values.reshape(cnt_units, num_time_steps_dec, num_treatments)
    current_treatments = current_treatments[:, :-offset, :] # (num_units, 1-94, 6). One-Hot-encoded treatments
    previous_treatments = current_treatments[:, :-1, :] # (num_units, 1-93, 6)

    outputs = df['outcome'].values.reshape(cnt_units, num_time_steps_dec, num_outputs) 
    outputs = outputs[:, horizon:, :] # (num_units, 2-95 timesteps, 1)
    
    active_entries = np.ones((cnt_units, num_time_steps_dec-1, 1)) # (num_units, 94, 1)
    active_entries[-(projection_horizon-1):] = 0
    sequence_lengths = (num_time_steps-1) * np.ones(cnt_units) #(num_time_steps_dec-1) * np.ones(num_units)
    
    data = {"current_covariates": current_covariates, 
            "current_treatments": current_treatments,
            "previous_treatments": previous_treatments,
            "outputs": outputs,
            "active_entries": active_entries,
            "sequence_lengths": sequence_lengths
           }
    return data

In [16]:
train_obj_dec = process_train_val_decoder(train_obj_enc)
val_obj_dec = process_train_val_decoder(val_obj_enc)
test_seq_obj_dec = process_test_decoder(test)

In [17]:
for k in train_obj_dec.keys():
    print(k, train_obj_dec[k].shape)

current_covariates (3126, 89, 7)
current_treatments (3126, 89, 6)
previous_treatments (3126, 88, 6)
outputs (3126, 89, 1)
active_entries (3126, 89, 1)
sequence_lengths (3908,)


In [18]:
for k in test_seq_obj_dec.keys():
    print(k, test_seq_obj_dec[k].shape)

current_covariates (392, 94, 7)
current_treatments (392, 94, 6)
previous_treatments (392, 93, 6)
outputs (392, 94, 1)
active_entries (392, 94, 1)
sequence_lengths (3908,)


### Export pickle files

In [19]:
with open('../data/train_dec.p', 'wb') as f:
    pickle.dump(train_obj_dec, f, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('../data/val_dec.p', 'wb') as f:
    pickle.dump(val_obj_dec, f, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('../data/test_seq_dec.p', 'wb') as f:
    pickle.dump(test_seq_obj_dec, f, protocol=pickle.HIGHEST_PROTOCOL)

# Data for Inference

For each unit, we are interested in the future outcome for time steps 96-100 under 6 difference treatment plans: Treatment 0-0-0-0-0, ..., 5-5-5-5-5

## Encoder

In [114]:
inf_obj_enc = process_data_encoder(df.sort_values(by=['unitID', 'weekID']), num_time_steps=95)

In [115]:
for k in inf_obj_enc.keys():
    print(k, inf_obj_enc[k].shape)

current_covariates (3908, 94, 9)
current_treatments (3908, 94, 6)
previous_treatments (3908, 93, 6)
outputs (3908, 94, 1)
active_entries (3908, 94, 1)
sequence_lengths (3908,)


Note that for encoder- for each unit, the output (based on time steps 1-95) is the same for each sequence of treatments. Hence axis 0 shape is 3908 (num_units). 

For decoder, we need to prepare data for each sequence of treatments. Hence axis 0 shape will be 3908*6.

## Decoder

In [22]:
static_vars = ["X1", "E_2", "E_3", "E_4", "E_5", "E_6"]

In [23]:
def get_data_inference(df, num_infer_steps=5):
    ''' Append data for next 5 time steps (96-100) for each treatment (1-6) '''
    df_infer = pd.DataFrame()
    units = np.sort(df['unitID'].unique())

    for unit in tqdm(units):
        unit_df = df[df['unitID'] == unit]
        unit_info = unit_df.iloc[0]
        
        for treat in [0.0, 0.1, 0.2, 0.3, 0.4, 0.5]:
            # Append info vars
            df_temp = pd.DataFrame({"unitID": [unit_info["unitID"]]*num_infer_steps, 
                                   "weekID": list(range(95, 95+num_infer_steps)),
                                   "treatment": [treat] * num_infer_steps})
            # Repeat static vars
            for var in static_vars:
                df_temp = pd.concat([df_temp, pd.DataFrame({var: [unit_info[var]]*num_infer_steps})], axis=1)
            
            # Append data for unit i (time steps 1-95) with data for treatment k (time steps 96-100)
            df_infer = pd.concat([df_infer, unit_df.copy(), df_temp], axis=0)
            
    df_infer = df_infer.fillna(0)
    return df_infer

In [32]:
def process_infer_data_decoder(df, num_infer_steps=5):
    ''' Prepare data for decoder inference '''
    num_time_steps_inf = num_time_steps_dec + num_infer_steps # 95+5
    cnt_units = df['unitID'].nunique() * 6 # Duplicating inference data for each treatment
    # ^Time steps 1-99 reqd for each sequence of treatments.

    current_covariates = df[input_features_dec].values.reshape(cnt_units, num_time_steps_inf, len(input_features_dec))
    current_covariates = current_covariates[:, :-offset, :] # (num_units, 1-99 timesteps, op+_num_static_features)

    current_treatments = pd.get_dummies(
                            df['treatment']).values.reshape(cnt_units, num_time_steps_inf, num_treatments)
    current_treatments = current_treatments[:, :-offset, :] # (num_units, 1-99, 6). One-Hot-encoded treatments
    previous_treatments = current_treatments[:, :-1, :] # (num_units, 1-98, 6)

    outputs = df['outcome'].values.reshape(cnt_units, num_time_steps_inf, num_outputs) 
    outputs = outputs[:, horizon:, :] # (num_units, 2-95 timesteps, 1)
    
    active_entries = np.ones((cnt_units, num_time_steps_inf-1, 1)) # (num_units, 99, 1)
    active_entries[-(projection_horizon-1):] = 0
    sequence_lengths = (num_time_steps_dec-1) * np.ones(cnt_units) #(num_time_steps_inf-1) * np.ones(num_units)
    
    data = {"current_covariates": current_covariates, 
            "current_treatments": current_treatments,
            "previous_treatments": previous_treatments,
            "outputs": outputs,
            "active_entries": active_entries,
            "sequence_lengths": sequence_lengths
           }
    return data

In [25]:
df_infer = get_data_inference(df)

100%|███████████████████████████████████████| 3908/3908 [10:03<00:00,  6.48it/s]


In [26]:
df_infer.to_csv("../data/df_infer.csv.gz", index=False, compression="gzip")
len(df_infer)

2344800

In [33]:
inf_seq_obj_dec = process_infer_data_decoder(df_infer)

In [35]:
for k in inf_seq_obj_dec.keys():
    print(k, inf_seq_obj_dec[k].shape)

current_covariates (23448, 99, 7)
current_treatments (23448, 99, 6)
previous_treatments (23448, 98, 6)
outputs (23448, 99, 1)
active_entries (23448, 99, 1)
sequence_lengths (23448,)


In [30]:
df['unitID'].nunique() * 6

23448

### Export pickle files

In [116]:
with open('../data/inf_enc.p', 'wb') as f:
    pickle.dump(inf_obj_enc, f, protocol=pickle.HIGHEST_PROTOCOL)

with open('../data/inf_seq_dec.p', 'wb') as f:
    pickle.dump(inf_seq_obj_dec, f, protocol=pickle.HIGHEST_PROTOCOL)